## Задание 1. Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

Структура таблицы:
    - st_id - ID ученика
    - timest - Время решения карточки
    - correct - Правильно ли решена горошина?
    - subject - Дисциплина, в которой находится горошина
   
#### Если под усердными студентами мы понимаем студента, который правильно решил 20 задач за текущий месяц, т.е. выполнил всего 20 задач за месяц, то запрос в SQL будет такой: 

In [ ]:
SELECT
    COUNT(DISTINCT st_id) AS id_count
FROM
    (SELECT st_id, COUNT(correct) AS cor   
    FROM peas
    GROUP BY st_id
    HAVING cor == 20)

#### Если под усердными студентами мы понимаем студента, который правильно решил хотябы 20 задач за текущий месяц, т.е. выполнил 20 задач и более за месяц, то запрос в SQL будет с изменением в условии HAVING, вот такое: 

In [ ]:
SELECT
    COUNT(DISTINCT st_id) AS id_count
FROM
    (SELECT st_id, COUNT(correct) AS cor
    FROM peas
    GROUP BY st_id
    HAVING cor >= 20)

## Задание 2. Оптимизация воронки. Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

    - ARPU 
    - ARPAU 
    - CR в покупку 
    - СR активного пользователя в покупку 
    - CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
    - ARPU считается относительно всех пользователей, попавших в группы.

- Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
- Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

#### Структура таблиц:
 
- Таблица peas:
    - st_id - ID ученика
    - timest - Время решения карточки
    - correct - Правильно ли решена горошина?
    - subject - Дисциплина, в которой находится горошина


- Таблица studs:
    - st_id - ID ученика
    - test_grp -  Метка ученика в данном эксперименте
    

- Таблица final_project_check:
    - st_id - ID ученика
    - sale_time - Время покупки
    - money - Цена, по которой приобрели данный курс
    - subject - Дисциплина, в которой находится горошина

## Решение:

In [ ]:
WITH join_table as
    (SELECT
        l.correct as correct,
        l.st_id as st_id,
        l.subject as subject,
        l.timest as timest,
        r.money as money,
        r.sale_time as sale_time,
        d.test_grp  as test_grp
    FROM peas as l
    LEFT JOIN final_project_check as r
    ON l.st_id == r.st_id
    LEFT JOIN studs as d
    ON l.st_id == d.st_id),
    
arpu_table as
    (SELECT ROUND(money_sum / value, 2) as arpu, value
    FROM
        (SELECT
            COUNT(st_id) as value,
            SUM(money) as money_sum
        FROM join_table)),
    
arpau_table as
    (SELECT ROUND(purchases / id, 2) as arpau
    FROM 
        (SELECT
            COUNT(aa) as id, 
            SUM(bb) as purchases
        FROM
            (SELECT
                COUNT(st_id) as aa,
                COUNT(correct) as ss,
                SUM(money) as bb
            FROM
                join_table
            GROUP BY st_id, money, correct
            HAVING money > 0
            ORDER BY st_id
            )
        HAVING ss > 10)),

cr_table as
    (SELECT id_bye 
    FROM    
        (SELECT
            COUNT(st_id) as id_bye
        FROM join_table
        WHERE money > 0)),

cr_activ_table AS
    (SELECT COUNT(st_id) as id
    FROM    
        (SELECT
            st_id,
            COUNT(correct) as ss
        FROM join_table
        GROUP BY st_id
        HAVING ss > 10)),

cr_math_table AS
    (SELECT COUNT(st_id) as id_math
    FROM    
        (SELECT
            st_id,
            COUNT(correct) as ss,
            money
        FROM join_table
        WHERE subject = 'Math' AND money != 0
        GROUP BY st_id, subject, money
        HAVING ss >= 2))

SELECT arpau, arpu, 
    ROUND(id_bye / value * 100, 2) as cr, 
    ROUND(id / id_bye * 100, 2) as cr_activ_user,
    ROUND(id_math / value * 100, 2) as cr_math
FROM 
    arpau_table, 
    arpu_table, 
    cr_table, 
    cr_activ_table,
    cr_math_table